In [1]:
import copy
import random
from collections import defaultdict
from functools import reduce
import numpy as np

def probability(p):
    return p > random.uniform(0.0, 1.0)

def event_values(event, variables):
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple([event[var] for var in variables])

In [2]:
###Resourced from aima-python### 

from itertools import product

class BayesNet:
    """Bayesian network containing only boolean-variable nodes."""

    def __init__(self, node_specs=None):
        """Nodes must be ordered with parents before children."""
        self.nodes = []
        self.variables = []
        node_specs = node_specs or []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """Add a node to the net. Its parents must already be in the
        net, and its variable must not."""
        node = BayesNode(*node_spec)
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        self.nodes.append(node)
        self.variables.append(node.variable)
        for parent in node.parents:
            self.variable_node(parent).children.append(node)

    def variable_node(self, var):
        """Return the node for the variable named var.
        >>> burglary.variable_node('Burglary').variable
        'Burglary'"""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception("No such variable: {}".format(var))

    def variable_values(self, var):
        """Return the domain of var."""
        return [True, False]

    def __repr__(self):
        return 'BayesNet({0!r})'.format(self.nodes)

class BayesNode:
    """A conditional probability distribution for a boolean variable,
    P(X | parents). Part of a BayesNet."""
    def __init__(self, X, parents, cpt):
        """X is a variable name, and parents a sequence of variable
        names or a space-separated string.  cpt, the conditional
        probability table, takes one of these forms:

        * A number, the unconditional probability P(X=true). You can
          use this form when there are no parents.

        * A dict {v: p, ...}, the conditional probability distribution
          P(X=true | parent=v) = p. When there's just one parent.

        * A dict {(v1, v2, ...): p, ...}, the distribution P(X=true |
          parent1=v1, parent2=v2, ...) = p. Each key must have as many
          values as there are parents. You can use this form always;
          the first two are just conveniences.

        In all cases the probability of X being false is left implicit,
        since it follows from P(X=true).

        >>> X = BayesNode('X', '', 0.2)
        >>> Y = BayesNode('Y', 'P', {T: 0.2, F: 0.7})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})"""

        if isinstance(parents, str):
            parents = parents.split()

        # We store the table always in the third form above.
        if isinstance(cpt, (float, int)):  # no parents, 0-tuple
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # one parent, 1-tuple
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v,): p for v, p in cpt.items()}

        assert isinstance(cpt, dict)
        for vs, p in cpt.items():
            assert isinstance(vs, tuple) and len(vs) == len(parents)
            assert all(isinstance(v, bool) for v in vs)
            assert 0 <= p <= 1

        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability
        P(X=value | parents=parent_values), where parent_values
        are the values of parents in event. (event must assign each
        parent a value.)
        >>> bn = BayesNode('X', 'Burglary', {T: 0.2, F: 0.625})
        >>> bn.p(False, {'Burglary': False, 'Earthquake': True})
        0.375"""
        assert isinstance(value, bool)
        ptrue = self.cpt[event_values(event, self.parents)]
        return ptrue if value else 1 - ptrue

    def sample(self, event):
        return probability(self.p(True, event))

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

In [3]:
def joint_probability(network, **query):
    """
    Evaluate the joint probability of all variables in the network
    subject to the given query. query is a dict with variable names(nodes) as keys and 
    observed values as values(boolean).
    """
    variables = network.variables
    """
    Generate all possible combinations of boolean values for every variable
    [False False False False False False False]
    [False False False False False False True]
    """  
    combinations = product(*[network.variable_values(variable) for variable in variables])

    joint_prob = 0
    for combination in combinations:
        #Pattern validity 
        valid = True
        for var_name, var_value in query.items():
            #Compare the combination and query dict and check if they are equal or not
            if combination[variables.index(var_name)] != var_value:
                valid = False
                break 
        if valid:
            prob = 1.0
            """
            Checks every node and its parent and returns the conditional probability 
            from the CPT using Bayesian Network class which calls teh Bayesnode class. 
            """
            for node in network.nodes:
                #returns the probability value of the parent variable from the CPTs.
                cpt = node.cpt[tuple(combination[variables.index(parent)] for parent in node.parents)]
                #checks for the value of the variable, returns 1-cpt if not.
                prob *= cpt if combination[variables.index(node.variable)] else 1 - cpt
            joint_prob += prob
    return joint_prob

In [6]:
# 0 -> False, 1 -> True
T, F = True, False
network = BayesNet([
    ('A', '', 0.28),
    ('B', 'A', {F: 0.65, T: 0.45}),
    ('C', 'B A', {(F, F): 0.21,(F, T): 0.14, (T, F): 0.65, (T, T): 0.71}),
    ('D', 'C', {F: 0.7, T: 0.51}),
    ('E', 'D C', {(F, F): 0.91,(F, T): 0.87, (T, F): 0.71, (T, T): 0.4}),
    ('F', 'E', {F: 0.92, T: 0.88}),
    ('G', 'F', {F: 0.64, T: 0.72})
])

In [7]:
#example query1
query = {'A': False, 'B': True, 'C': True, 'D': True, 'E':False} #A=F,B=T,C=T,D=T,E=F
joint_prob = joint_probability(network, **query)
print(joint_prob)

0.09308519999999998


In [8]:
#example query2
query = {'C': True} #C=T
joint_prob = joint_probability(network, **query)
print(joint_prob)

0.4681399999999999


In [9]:
#example query3
query = {'C': False, 'D': True, 'E': True} #C=F,D=T,E=T
joint_prob = joint_probability(network, **query)
print(joint_prob)

0.26433442


In [11]:
#Couple of some example hypothesis
queries = [{'A': False, 'B': True, 'C': True},
           {'D': True, 'C': True},
           {'E': True, 'C': True, 'D': True},
           {'E': True, 'F': True, 'G': True}
           ]
for query in queries:
  joint_prob = joint_probability(network, **query)
  print(joint_prob)

0.30419999999999997
0.2387514
0.09550056
0.4464350934912
